In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import autoencoder as ae
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot

path = "/media/jahan/solo/"
os.chdir(path)

# prep data
labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
normal = labels[labels["No Finding"] == 1]
normal = normal.Path
abnormal = labels[labels["No Finding"] != 1]
abnormal = abnormal.Path
print(len(normal),len(abnormal))

x_a = pd.concat([normal,abnormal]).values
indexes_switch = np.random.choice(len(x_a), int(len(x_a)/2), replace=False)
x_b = pd.concat([normal.sample(frac=1),abnormal.sample(frac=1)]).values
x_c = pd.concat([abnormal.sample(n=len(normal)),normal.sample(n=len(abnormal),replace=True)]).values

temp_b = x_b[indexes_switch]
temp_c = x_c[indexes_switch]
x_b[indexes_switch] = temp_c
x_c[indexes_switch] = temp_b

y = np.full((len(x_a),2),[0,1])
y[indexes_switch] = np.full((len(indexes_switch),2),[1,0])


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/"


encoder = ae(a_paths=x_a, b_paths=x_b, c_paths=x_c, labels=y, save_path=save_path)
#encoder.build_dataset(batch_size=32)
encoder.build()

16974 174053


W0814 22:24:10.872467 140493952886592 deprecation.py:506] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 32          input_1[0][0]                    
                                                                 input_2[0][0]                

In [ ]:
encoder.train(10)

W0814 22:24:21.263019 140493952886592 deprecation.py:323] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
1560/5000 [========>.....................] - ETA: 11:41 - loss: 0.7469 - auc: 0.6855 - false_negatives: 18138.0000 - false_positives: 17746.0000 - true_negatives: 32174.0000 - true_positives: 31782.0000